In [165]:
import numpy as np
import importlib
import matplotlib.pyplot as plt
import sklearn.decomposition as decomposition
import sklearn.ensemble as ensemble
import sklearn.svm as svm

In [151]:
import dataGenerator
importlib.reload(dataGenerator)

<module 'dataGenerator' from 'C:\\Users\\18pep\\Documents\\sem_I\\DVaE\\clustering_dimensions_reduction\\dataGenerator.py'>

In [147]:
X,y= (dataGenerator.dataGenerator(500)
     .generate_relevant(0,2.5,1,1.5,5)
     .generate_irrelevant(2,5)
     .generate_correlated(5)
     .generate_redundant(5)
     .build())

In [159]:
X[0]

array([-0.67004481, -0.98174288, -0.11897465, -0.60471777, -1.42798813,
        1.1123472 ,  0.015864  ,  0.5499582 ,  1.2502887 ,  0.12547627,
       -0.9233141 ,  0.49019279, -0.97249034, -0.80085356, -1.24429805,
       -0.60471777, -0.16679261,  0.83228651, -0.67004481, -0.16679261])

array([[-2.79320187e+01, -6.07617389e+00, -5.31864327e+00,
         6.26491239e-01, -9.24235186e-01],
       [-2.78909200e+01, -5.48867073e+00, -4.03364110e+00,
        -1.34036189e-01, -2.50840023e-01],
       [-2.85884007e+01, -5.42245716e+00, -3.13617581e+00,
         8.10309051e-01,  1.34366830e+00],
       ...,
       [-2.60753536e+01, -2.37145961e-01,  5.79463228e+00,
        -1.72921412e+00,  5.79166255e+00],
       [ 1.49671951e+02,  4.37165761e+01, -2.59603220e+00,
        -3.78113039e+00,  1.00472226e+00],
       [ 2.09332693e+02,  6.00620968e+01, -8.43476992e+00,
        -1.89871742e-01,  5.77280077e-01]], shape=(500, 5))